In [10]:
import duckdb
import pandas as pd

### Run on in-memory database

In [11]:
# run queries using an in-memory database
test = duckdb.sql("select 'French' as Nationality, 'Marie' as Name, 24 as Age, ['tennis','soccer','golf'] as hobby")
print(test)

┌─────────────┬─────────┬───────┬────────────────────────┐
│ Nationality │  Name   │  Age  │         hobby          │
│   varchar   │ varchar │ int32 │       varchar[]        │
├─────────────┼─────────┼───────┼────────────────────────┤
│ French      │ Marie   │    24 │ [tennis, soccer, golf] │
└─────────────┴─────────┴───────┴────────────────────────┘



In [12]:
duckdb.sql('select hobby[1], hobby[2], hobby[3] from test')

┌──────────┬──────────┬──────────┐
│ hobby[1] │ hobby[2] │ hobby[3] │
│ varchar  │ varchar  │ varchar  │
├──────────┼──────────┼──────────┤
│ tennis   │ soccer   │ golf     │
└──────────┴──────────┴──────────┘

### Read local file as a table

In [13]:
# 1. use file path directly into from statement
duckdb.sql('select * from "data/health_survey.csv" limit 10')

┌────────────┬─────────────────────┬──────────────────┬────────┬─────────────────┬───────────────────────┬───────────────────────┬────────┬───────────────┬──────────────────┬────────┬────────────┬─────────┬─────────┬────────────┐
│ is_diabete │ high_blood_pressure │ high_cholesterol │  bmi   │      smoke      │       exercise        │        alcohol        │ gender │ mental_health │       edu        │ income │ employment │  race   │ flushot │   state    │
│   int64    │       double        │      double      │ double │     varchar     │        double         │        double         │ int64  │    double     │     varchar      │ double │   double   │ varchar │ double  │  varchar   │
├────────────┼─────────────────────┼──────────────────┼────────┼─────────────────┼───────────────────────┼───────────────────────┼────────┼───────────────┼──────────────────┼────────┼────────────┼─────────┼─────────┼────────────┤
│          0 │                 1.0 │              0.0 │ 2650.0 │ never_smoked   

In [14]:
# 2. load csv file and save it as a table
try_csv = duckdb.read_csv('data/health_survey.csv')
duckdb.sql('select * from try_csv limit 10')

┌────────────┬─────────────────────┬──────────────────┬────────┬─────────────────┬───────────────────────┬───────────────────────┬────────┬───────────────┬──────────────────┬────────┬────────────┬─────────┬─────────┬────────────┐
│ is_diabete │ high_blood_pressure │ high_cholesterol │  bmi   │      smoke      │       exercise        │        alcohol        │ gender │ mental_health │       edu        │ income │ employment │  race   │ flushot │   state    │
│   int64    │       double        │      double      │ double │     varchar     │        double         │        double         │ int64  │    double     │     varchar      │ double │   double   │ varchar │ double  │  varchar   │
├────────────┼─────────────────────┼──────────────────┼────────┼─────────────────┼───────────────────────┼───────────────────────┼────────┼───────────────┼──────────────────┼────────┼────────────┼─────────┼─────────┼────────────┤
│          0 │                 1.0 │              0.0 │ 2650.0 │ never_smoked   

In [15]:
# 3. read pandas dataframe as a table - read only
survey_df = pd.read_csv('data/health_survey.csv')
duckdb.sql('select * from survey_df limit 10')

┌────────────┬─────────────────────┬──────────────────┬────────┬─────────────────┬───────────────────────┬───────────────────────┬────────┬───────────────┬──────────────────┬────────┬────────────┬─────────┬─────────┬────────────┐
│ is_diabete │ high_blood_pressure │ high_cholesterol │  bmi   │      smoke      │       exercise        │        alcohol        │ gender │ mental_health │       edu        │ income │ employment │  race   │ flushot │   state    │
│   int64    │       double        │      double      │ double │     varchar     │        double         │        double         │ int64  │    double     │     varchar      │ double │   double   │ varchar │ double  │  varchar   │
├────────────┼─────────────────────┼──────────────────┼────────┼─────────────────┼───────────────────────┼───────────────────────┼────────┼───────────────┼──────────────────┼────────┼────────────┼─────────┼─────────┼────────────┤
│          0 │                 1.0 │              0.0 │ 2650.0 │ never_smoked   

In [16]:
# 3. read pandas dataframe as a table - read only
health_exp = pd.read_csv('data/US_PER_CAPITA20.CSV')
health_exp = health_exp.rename(columns={'Group':'group_'})

health_exp_tb = duckdb.sql("select Item, group_, State_Name, Y2020 from health_exp where group_ = 'State'").df()
health_exp_tb

,Item,group_,State_Name,Y2020
0,Personal Health Care ($),State,Alabama,9280
1,Personal Health Care ($),State,Alaska,13642
2,Personal Health Care ($),State,Arizona,8756
3,Personal Health Care ($),State,Arkansas,9338
4,Personal Health Care ($),State,California,10299
...,...,...,...,...
505,"Other Health, Residential, and Personal Care ($)",State,Virginia,491
506,"Other Health, Residential, and Personal Care ($)",State,Washington,714
507,"Other Health, Residential, and Personal Care ($)",State,West Virginia,637
508,"Other Health, Residential, and Personal Care ($)",State,Wisconsin,486


### Run on disk

In [17]:
# create db and save data on disk
con = duckdb.connect("data/diabetes_db.db")
con.sql('CREATE TABLE test (name varchar, age int, hobby varchar[])')
con.sql("INSERT INTO test VALUES ('Marie', 13, ['hockey','squash'])")
con.close()

In [18]:
with duckdb.connect('data/diabetes_db.db') as con:
    print('-- table list --')
    print(con.sql('show tables'))
    print('-- test table --')
    print(con.sql('select * from test'))

-- table list --
┌───────────────┐
│     name      │
│    varchar    │
├───────────────┤
│ health_exp_tb │
│ survey        │
│ test          │
└───────────────┘

-- test table --
┌─────────┬───────┬──────────────────┐
│  name   │  age  │      hobby       │
│ varchar │ int32 │    varchar[]     │
├─────────┼───────┼──────────────────┤
│ Marie   │    13 │ [hockey, squash] │
└─────────┴───────┴──────────────────┘



### add table of Pandas DataFrame into database 

In [ ]:
with duckdb.connect('data/diabetes_db.db') as conn:
    conn.sql('drop table test')
    conn.sql('CREATE TABLE survey AS (select * from survey_df)')
    conn.sql('CREATE TABLE health_exp_tb AS (select * from health_exp_tb)')
    print(conn.sql('show tables'))

┌───────────────┐
│     name      │
│    varchar    │
├───────────────┤
│ health_exp_tb │
│ survey        │
└───────────────┘

